# 03_08: SOLUTION - Query Your Knowledge Graph with Cypher

In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI

In [ ]:
load_dotenv(dotenv_path='../.env')

URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_USER")
PWD = os.getenv("NEO4J_PWD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

graph = Neo4jGraph(url=URI, username=USER, password=PWD)

In [ ]:
from langchain.document_loaders import WikipediaLoader

text = WikipediaLoader(query="skiing").load()

In [ ]:
text[0:100]

In [ ]:
len(text)

In [ ]:
extracted_info = [
    {
        "title": doc.metadata.get("title", ""),
        "summary": doc.metadata.get("summary", ""),
        "snippet": doc.page_content[:100]
    }
    for doc in text
]

pprint(extracted_info)


In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(text)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
graph.add_graph_documents(
    graph_documents,
    include_source=True
)